Importing libraries

In [8]:
# For fetching citybik API
import requests

Fetching the API

In [11]:
citybik_api_url = "http://api.citybik.es/v2/networks/to-bike"

response = requests.get(citybik_api_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:

    # Parse and use the response data (e.g., JSON)
    data = response.json()
else:
    # Handle the error
    print(f"Request failed with status code: {response.status_code}")

2. Count and print the number of active stations (a station is active if its extra.status field is "online").

In [29]:
active_stations = sum([1 if station['extra']['status'] == 'online' else 0 for station in data['network']['stations']])
print(f"Active stations: {active_stations}")

Active stations: 10


3. Count and print the total number of bikes available (field free_bikes) and the number of free docks
(field empty_slots) throughout all stations.

In [33]:
total_bikes = 0
total_empty_slots = 0

for station in data['network']['stations']:
    total_bikes += station['free_bikes']
    total_empty_slots += station['empty_slots']

print(f"Total bikes: {total_bikes}")
print(f"Total empty slots: {total_empty_slots}")   

Total bikes: 103
Total empty slots: 95


Given the coordinates (latitude, longitude) of a point (e.g. 45.074512, 7.694419), identify the
closest bike station to it that has available bikes. For computing the distance among two points
(given their coordinates), you can use the function distance_coords() defined in the code snippet
below (which is an implementation of the great-circle distance):
from math import cos, acos, sin
from math import cos, acos, sin

def distance_coords(lat1, lng1, lat2, lng2):
    """Compute the distance among two points."""
    deg2rad = lambda x: x * 3.141592 / 180
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))

In [35]:
from math import cos, acos, sin
def distance_coords(lat1, lng1, lat2, lng2):
    """Compute the distance among two points."""
    deg2rad = lambda x: x * 3.141592 / 180
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))

Mode 1

In [49]:
given_coords = (45.074512, 7.694419)
arg_min = 0
min_distance = distance_coords(given_coords[0], given_coords[1], data['network']['stations'][0]['latitude'], data['network']['stations'][0]['longitude'])
for i, station in enumerate(data['network']['stations']):
    current_distance = distance_coords(given_coords[0], given_coords[1], station['latitude'], station['longitude'])
    if min_distance > current_distance:
        arg_min = i
        min_distance = distance_coords(given_coords[0], given_coords[1], station['latitude'], station['longitude'])

data['network']['stations'][arg_min]

{'empty_slots': 4,
 'extra': {'number': 10,
  'reviews': 40,
  'score': 4.5,
  'status': 'online',
  'uid': '592'},
 'free_bikes': 0,
 'id': 'f16cba9c6dc917e833eab7bf3e0168b7',
 'latitude': 45.114828943536864,
 'longitude': 7.641313076019287,
 'name': '10. Gallo Praile',
 'timestamp': '2023-10-17T08:15:06.190000Z'}

Mode 2

In [42]:
min(data['network']['stations'], key=lambda station: distance_coords(given_coords[0], given_coords[1], station['latitude'], station['longitude']))

{'empty_slots': 4,
 'extra': {'number': 10,
  'reviews': 40,
  'score': 4.5,
  'status': 'online',
  'uid': '592'},
 'free_bikes': 0,
 'id': 'f16cba9c6dc917e833eab7bf3e0168b7',
 'latitude': 45.114828943536864,
 'longitude': 7.641313076019287,
 'name': '10. Gallo Praile',
 'timestamp': '2023-10-17T08:15:06.190000Z'}